In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Download the data into pandas dataframe
!wget -O spacex_launch_dash.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                       # Create an division for adding dropdown helper text for choosing Launch Site
                                        dcc.Dropdown(
                                            id='site-dropdown', 
                                            # Update dropdown values using list comphrehension
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                                    ],
                                                     searchable=True, placeholder="Select a Launch Site Here" 
                                                    ),
                                        ]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                    dcc.RangeSlider(
                                        id='payload-slider',
                                        min=0,
                                        max=10000,
                                        step=1000,
                                        value=[min_payload, max_payload],
                                        marks={
                                            0: '0',
                                            2500: '2500',
                                            5000: '5000',
                                            7500: '7500',
                                            10000: '10000'
                                                }),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


                               
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)

# Add computation to callback function and return pie graph
def update_pichart(selected_launchsite):
    # For All Sites
    if selected_launchsite == 'ALL':
        fig_pie = px.pie(spacex_df, values='class', names='Launch Site', title="Total successful launches count for All Sites")
        
        return fig_pie
    # For selected site
    else:
        select_df = spacex_df.loc[spacex_df['Launch Site'] == selected_launchsite]
        fig_pie = px.pie(select_df, names='class', title="Total successful launches count for "+ selected_launchsite)
                
        return fig_pie
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)

def update_scatter(site,payload):
    # Selected payload from slider
    low,high = (payload[0], payload[1])
    # Filtering data frame form selected payload
    payload_df = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)] 
    # For All Site
    if site == 'ALL':
        fig_scatter = px.scatter(payload_df,x="Payload Mass (kg)", y="class", symbol = "Launch Site",  color="Booster Version Category", title='Payload vs. Outcome for All Sites')
    
        return fig_scatter
    # For Selected site
    else:
    # Use payload_df to filter by selected site
        site_df = payload_df[payload_df['Launch Site'] == site]
        fig_scatter = px.scatter(site_df,x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload vs. Outcome for' + site)
    
        return fig_scatter

# Run the app
if __name__ == '__main__':
    #Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="external", host="localhost", port=7645, debug=False, dev_tools_ui=False, dev_tools_props_check=False)
